In [3]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import orjson
import zipfile
from pathlib import Path

# Create a connection to the SQLite database
db = sqlite3.connect('hw1.db')
cursor = db.cursor()
cursor.execute('CREATE TABLE IF NOT EXISTS telecom_companies (inn WARCHAR(13), name WARCHAR(200), okved WARCHAR(20), kpp WARCHAR(9), ogrn WARCHAR(13))')
db.commit()

#counter = 0
sql_query = 'INSERT INTO telecom_companies (inn , name , okved , kpp , ogrn) VALUES (?, ?, ?, ?, ?)'

with zipfile.ZipFile('archive.zip', 'r') as zf:
    file_names = zf.namelist()
    for name in file_names:
        zf.extract(name)
        with open(name, 'r', encoding='utf8') as f:
            spr = orjson.loads(f.read()) 
        for itm in spr: 
            if 'СвОКВЭД' in itm['data']: 
                if 'СвОКВЭДОсн' in itm['data']['СвОКВЭД']:
                    okv = itm['data']['СвОКВЭД']['СвОКВЭДОсн']
                    if okv['КодОКВЭД'][:3] == '61.':
                        try:
                            cursor.execute(sql_query, [itm['inn'], itm['name'], okv['КодОКВЭД'], itm['kpp'], itm['ogrn']])
                        except:
                            print('Что-то пошло не так!')
                            db.close()
                            exit(1)
        path = Path(name)
        path.unlink()

db.commit()
cursor.close()
db.close()

engine = create_engine('sqlite:///hw1.db')
df = pd.read_sql_query('SELECT * FROM telecom_companies', engine)
print("Количество организаций в таблице:", len(df))
engine.dispose()

Количество организаций в таблице: 6
